In [2]:
# packages

from collections import defaultdict, OrderedDict, Counter
from datetime import datetime

from dateutil.relativedelta import relativedelta
import glob
from itertools import product, combinations
import matplotlib.pyplot as plt
import numpy    as np
#import networkx as nx
from operator import itemgetter
import pandas   as pd
import rdflib as rdflib
import re
from scipy.signal import convolve2d
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer, minmax_scale
from SPARQLWrapper import SPARQLWrapper, JSON, POST, TURTLE
import sys
from rdflib.namespace import XSD, Namespace
from rdflib.term import URIRef

import IPython
import pprint
pp = pprint.PrettyPrinter(indent=4)



from constants import *
from helpers import *

In [3]:
# helpers

# tuples
end_year=2022
start_year=1971
period_length=5
year_range = end_year - start_year
modulo = year_range % period_length
if modulo == 0:
    final_start = end_year - period_length
else:
    final_start = end_year - modulo
final_end = end_year + 1

starts = np.arange(start_year, final_start, period_length).tolist()
tuples = [(start, start+period_length) for start in starts]# five year periods
# make 5-year bins


# We'll add the last period calculated earlier
tuples.append(tuple([final_start, final_end]))
bins = pd.IntervalIndex.from_tuples(tuples, closed='left')
original_labels = list(bins.astype(str))
new_labels = ['{} - {}'.format(b.strip('[)').split(', ')[0], int(b.strip('[)').split(', ')[1])-1) for b in original_labels]
label_dict = dict(zip(original_labels, new_labels))

# add period function
def add_period(df_fo, columnname="year"):
    df_fo['period'] = pd.cut(df_fo[columnname], bins=bins, include_lowest=True, precision=0)
    df_fo['period'] = df_fo['period'].astype("str")

    df_fo["period"]=df_fo["period"].apply(lambda x: int(x.split(",")[0].replace("[","")))
    return df_fo

# decade function
def add_decade(df_fi, columname="year"):
    df_fi["decade"]=(df_fi["year"]-1)- ((df_fi["year"]-1)%10)
    
    return df_fi

In [4]:
df_fi=pd.read_csv("../../data/df_fi_all.csv", sep="\t", index_col=0)
df_fo=pd.read_csv("../../data/df_fo_all2.csv", sep="\t", index_col=0)

In [ ]:
# translated graph
from rdflib import Graph
g = Graph() 
g.parse('../../data/nat_lang_graph2.nt')

# finnish graph

f = Graph() 

f.parse('../../data/finnish_graph_new.nt')

In [ ]:
# compare fis

df_fi_old=pd.read_csv("../../data/df_fi_all.csv", sep="\t", index_col=0)
old_novels=[str(n) for n in df_fi_old["novel"]]
mystery_novels=[d for d in df_fi.novel.unique() if d not in old_novels]

In [ ]:
print(len(mystery_novels))
print(len(df_fi), len(df_fi_old), len(df_fi))

In [ ]:
MY_VALUES = ["<"+str(n)+">" for n in mystery_novels]
valuestring=" ".join(MY_VALUES)

In [ ]:
# query mystery_novels

# query labels
Q ="""
SELECT  DISTINCT  *  WHERE {

?novel rdf:type ?class .
VALUES ?novel { MY_VALUES }
?novel kaunokki:alkukieli ?lang .
?novel skos:prefLabel ?nimi . FILTER(LANG(?nimi)="fi")
?novel kaunokki:manifests_in ?pub .
?pub kaunokki:ilmestymisvuosi ?vuosi .
?vuosi skos:prefLabel ?year .

#FILTER NOT EXISTS { xsd:int(?year) < 1970 }

} 
    

"""

#sparql = SPARQLWrapper("http://ldf.fi/kirjasampo/sparql")

#nat_labels=pd.DataFrame()
#for nat in df_fo2["nat"].unique():
sparql = SPARQLWrapper("http://ldf.fi/booksampo-2022/sparql")
q=Q.replace("MY_VALUES", valuestring)
sparql.setQuery(PREFIXES + q)
sparql.setReturnFormat(JSON)
sparql.addCustomHttpHeader(*list(AUTHORIZATION_HEADER.items())[0])
results = sparql.query().convert()
temp = JSON2Pandas2(results)

    



In [ ]:
temp.lang.unique()
temp[temp["lang"]=='http://lexvo.org/id/iso639-3/eng']
temp["year"]= pd.to_numeric(temp["year"])
temp[temp["year"]<1971]
temp["class"].unique()

In [ ]:
temp.head(20)

In [ ]:
df_fi.sample()